<a href="https://colab.research.google.com/github/Maya7991/gsc_classification/blob/main/Net1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gtts pydub --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.6 MB/s eta 0:00:00


In [ ]:
import torch, os
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch import nn

from torchaudio.transforms import MelSpectrogram, AmplitudeToDB
from glob import glob
import torchaudio

from gtts import gTTS
from pydub import AudioSegment
import random
import shutil
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
DIR = "/content/drive/MyDrive/data"
os.makedirs("data/wake", exist_ok=True)
os.makedirs("data/background", exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
for i in range(25):
    tts = gTTS("Mocha")
    tts.save(f"data/wake/mocha_{i}.mp3")
    sound = AudioSegment.from_mp3(f"data/wake/mocha_{i}.mp3")
    sound = sound.set_frame_rate(16000).set_channels(1)
    sound.export(f"data/wake/mocha_{i}.wav", format="wav")
    os.remove(f"data/wake/mocha_{i}.mp3")

# Download background samples from Google Speech Commands v0.02
# You can download manually or script:
# wget http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz

# After extracting:
# Use random non-"mocha" samples
SOURCE = f"{DIR}/speech_commands/"
KEYWORDS = ["yes", "no", "down", "left", "right", "off", "go"]

for kw in tqdm(KEYWORDS):
    files = os.listdir(os.path.join(SOURCE, kw))
    selected = random.sample(files, 15)
    for f in selected:
        shutil.copy(os.path.join(SOURCE, kw, f), f"data/background/{kw}_{f}")


100%|██████████| 7/7 [01:31<00:00, 13.07s/it]


In [ ]:
class AudioCNN(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.AdaptiveAvgPool2d((1, 1))
        )
        self.fc = nn.Linear(64, n_classes)

    def forward(self, x):
        x = self.net(x)
        return self.fc(x.view(x.size(0), -1))


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

class WakeWordDataset(Dataset):
    def __init__(self, wake_dir, background_dir):
        self.samples = []
        self.labels = []
        self.label_map = {"wake": 1, "background": 0}
        self.mel = MelSpectrogram(16000, n_mels=64)
        self.db = AmplitudeToDB()

        for f in glob(f"{wake_dir}/*.wav"):
            self.samples.append(f)
            self.labels.append(1)

        for f in glob(f"{background_dir}/*.wav"):
            self.samples.append(f)
            self.labels.append(0)

    def __len__(self): return len(self.samples)

    def __getitem__(self, idx):
        x, sr = torchaudio.load(self.samples[idx])
        if sr != 16000:
            x = torchaudio.functional.resample(x, sr, 16000)
        x = self.db(self.mel(x)).clamp(min=-80, max=0) / 80 + 1  # normalize to [0,1]
        return x, self.labels[idx]



In [ ]:
if __name__ == "__main__":
    ds = WakeWordDataset("data/wake", "data/background")
    dl = DataLoader(ds, batch_size=16, shuffle=True)
    for x, y in dl:
        print(x.shape, y.shape)
        break
    model = AudioCNN(n_classes=2).to(device)
    opt = torch.optim.Adam(model.parameters(), lr=1e-3)

    for epoch in range(10):
        model.train()
        total, correct = 0, 0
        for xb, yb in dl:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            loss = F.cross_entropy(out, yb)
            opt.zero_grad(); loss.backward(); opt.step()
            correct += (out.argmax(1) == yb).sum().item()
            total += yb.size(0)
        print(f"Epoch {epoch+1}, Accuracy: {correct/total:.2%}")

    torch.save(model.state_dict(), "wakeword_mocha.pt")

RuntimeError: stack expects each tensor to be equal size, but got [1, 64, 81] at entry 0 and [1, 64, 72] at entry 7